# Data Modelling

## Get the data from the previous notebook

##### Import the libraries

In [ ]:
import json
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings("ignore")

##### Get the data from the previous notebook, as csv files, and load them into dataframes

In [ ]:
USE_MERGED_DATA = True

awards_players_df = pd.read_csv('../prep_data/awards_players_df.csv')
coaches_df = pd.read_csv('../prep_data/coaches_df.csv')
players_df = pd.read_csv('../prep_data/players_df.csv')
players_teams_df = pd.read_csv('../prep_data/players_teams_df.csv')
series_post_df = pd.read_csv('../prep_data/series_post_df.csv')
if USE_MERGED_DATA:
    teams_df = pd.read_csv('../prep_data/prepared_dataset.csv')
else:
    teams_df = pd.read_csv('../prep_data/teams_df.csv')
teams_post_df = pd.read_csv('../prep_data/teams_post_df.csv')

# Make a dictionary with all the dataframes
dfs = {'awards_players_df': awards_players_df, 'coaches_df': coaches_df, 'players_df': players_df, 'players_teams_df': players_teams_df, 'series_post_df': series_post_df, 'teams_df': teams_df, 'teams_post_df': teams_post_df}

##### Get the data from competition year

In [ ]:
# DB Credentials
with open("../config.json") as config_file:
    config = json.load(config_file)

host = config["db_host"]
user = config["db_user"]
password = config["db_password"]
database = config["db_database"]
schema = config["db_11_schema"]

In [ ]:
connection = psycopg2.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

cursor = connection.cursor()

def execute(query):
    cursor.execute(query)
    connection.commit()
    return cursor.fetchall()

def fetch(query):
    cursor.execute(query)
    return cursor.fetchall()

SELECT = "SELECT * FROM " + schema + "." # + table_name 
INSERT = "INSERT INTO " + schema + "." # + table_name + " VALUES " + values
UPDATE = "UPDATE " + schema + "." # + table_name + " SET " + column_name + " = " + value
DELETE = "DELETE FROM " + schema + "."  # + table_name + " WHERE " + column_name + " = " + value

In [ ]:
coaches = fetch(SELECT + "coaches") # all coaches who've managed the teams during the time period,
players_teams = fetch(SELECT + "players_teams") # performance of each player for each team they played,
teams = fetch(SELECT + "teams") # performance of the teams for each season,

players_teams_11_df = pd.DataFrame(players_teams, columns=['playerID', 'year', 'stint', 'tmID', 'lgID'])
teams_11_df = pd.DataFrame(teams, columns=['year', 'lgID', 'tmID', 'franchID', 'confID', 'name', 'arena', 'playoff'])
coaches_11_df = pd.DataFrame(coaches, columns=['coachID', 'year', 'tmID', 'lgID', 'stint'])

##### Make some preprocessing to year 11 data to make it compatible with the other years

In [ ]:
binary_columns = ['confID', 'playoff']

for col in binary_columns:
    teams_11_df[col] = teams_11_df[col].replace('EA', 0)
    teams_11_df[col] = teams_11_df[col].replace('WE', 1)
    teams_11_df[col] = teams_11_df[col].replace('N', 0)
    teams_11_df[col] = teams_11_df[col].replace('Y',1)


players_teams_11_df = players_teams_11_df.reindex(columns=players_teams_df.columns)
players_teams_df = pd.concat([players_teams_11_df, players_teams_df])

teams_11_df = teams_11_df.reindex(columns=teams_df.columns)
teams_df = pd.concat([teams_11_df, teams_df])

coaches_11_df = coaches_11_df.reindex(columns=coaches_df.columns)
coaches_df = pd.concat([coaches_11_df, coaches_df])

# MODELING

### Imports, constants and functions

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.model_selection import GridSearchCV

Define some constants that we will be going to use across the notebook

In [ ]:
TARGET_NAMES = ['No Playoffs', 'Playoffs']
N_FOLDS_CV = 10

##### Create reusable functions that will be useful across the notebook

In [ ]:
def print_confusion_matrix(cm, target_names):
    df_cm = pd.DataFrame(cm, index=target_names, columns=target_names)
    sns.heatmap(df_cm, annot=True, cmap='Blues', fmt='g')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

def print_results(clf, X_train, X_test, y_train, y_test, output=True):
    # Fit the classifier on the training data
    clf.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = clf.predict(X_test)

    # Calculate the accuracy of the classifier
    accuracy = accuracy_score(y_test, y_pred)
    if output:
        print("Accuracy:", accuracy)

    # Create a confusion matrix to evaluate the model and print with labels
    cm = confusion_matrix(y_test, y_pred)

    if output:
        print_confusion_matrix(cm, TARGET_NAMES)

    return y_pred

def cross_validation(clf, X, y, output=True):
    cv = StratifiedKFold(n_splits=N_FOLDS_CV, shuffle=True, random_state=42)
    scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')
    
    # Display the cross-validation results
    if output:
        print("Cross-Validation Results:")
        print("Mean Accuracy: {:.2f}%".format(scores.mean() * 100))
        print("Standard Deviation: {:.2f}".format(scores.std()))

    return scores
    
def force_qualify_8_teams(y_pred, y_test, confIDs, output=True):  
    # Split the predictions and test labels by conference
    y_pred_conf0 = y_pred[confIDs == 0]
    y_pred_conf1 = y_pred[confIDs == 1]
    y_test_conf0 = y_test[confIDs == 0]
    y_test_conf1 = y_test[confIDs == 1]

    # Get the indices of the top 4 teams in each conference
    top_teams_indices_conf0 = y_pred_conf0.argsort()[-4:][::-1]
    top_teams_indices_conf1 = y_pred_conf1.argsort()[-4:][::-1]

    # Convert the labels to numeric format (0 for 'No Playoff', 1 for 'Playoff')
    y_pred_numeric_conf0 = [1 if i in top_teams_indices_conf0 else 0 for i in range(len(y_pred_conf0))]
    y_pred_numeric_conf1 = [1 if i in top_teams_indices_conf1 else 0 for i in range(len(y_pred_conf1))]

    # Concatenate the predictions and test labels for both conferences
    y_pred_numeric = np.concatenate([y_pred_numeric_conf0, y_pred_numeric_conf1])
    y_test = np.concatenate([y_test_conf0, y_test_conf1])

    # Calculate the accuracy of the modified predictions
    accuracy = accuracy_score(y_test, y_pred_numeric)
    if output:
        print("Modified Accuracy (Top 4 Teams from Each Conference as Playoff):", accuracy)
    
    # Create a confusion matrix to evaluate the model and print with labels
    cm = confusion_matrix(y_test, y_pred_numeric)
    if output:
        print_confusion_matrix(cm, TARGET_NAMES)

## With current year info prediction

Run a simple model on the teams table to predict if a team will make it to the playoffs or not. Let's see how it behaves.

The main objective of this chapter is to find which is the model that performs the best, given the perfect data (the data from the year to be predicted)

In [ ]:
# Split data into train years and test year (year 10)
teams_train_df = teams_df[teams_df['year'] < 10]
teams_test_df = teams_df[teams_df['year'] == 10]

cross_val_scores = []

### Random Forest

In [ ]:
y_train = teams_train_df['playoff']
y_test = teams_test_df['playoff']
confIDs = teams_test_df['confID']
X_train = teams_train_df.drop(['playoff', 'year', 'tmID', 'firstRound', 'semis', 'finals'], axis=1)
X_test = teams_test_df.drop(['playoff', 'year', 'tmID', 'firstRound', 'semis', 'finals'], axis=1)

if USE_MERGED_DATA:
    X_train = X_train.drop(['W_post', 'L_post'], axis=1)
    X_test = X_test.drop(['W_post', 'L_post'], axis=1)

X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])

clf = RandomForestClassifier()

y_pred = print_results(clf, X_train, X_test, y_train, y_test)

In [ ]:
force_qualify_8_teams(y_pred, y_test, confIDs)

scores = cross_validation(clf, X, y)
cross_val_scores.append(scores)

### KNN

In [ ]:
knn_classifier = KNeighborsClassifier(n_neighbors=5)

y_pred = print_results(knn_classifier, X_train, X_test, y_train, y_test)

force_qualify_8_teams(y_pred, y_test, confIDs)

scores = cross_validation(knn_classifier, X, y)
cross_val_scores.append(scores)

### Multinomial NaiveBayes

In [ ]:
mnb_classifier = MultinomialNB()

y_pred = print_results(knn_classifier, X_train, X_test, y_train, y_test)

force_qualify_8_teams(y_pred, y_test, confIDs)

scores = cross_validation(mnb_classifier, X, y)
cross_val_scores.append(scores)

### Neural Network

In [ ]:
mlp_classifier = MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=1000)
    
y_pred = print_results(mlp_classifier, X_train, X_test, y_train, y_test)

force_qualify_8_teams(y_pred, y_test, confIDs)

scores = cross_validation(mlp_classifier, X, y)
cross_val_scores.append(scores)

### Conclusions

With the current year information, the best model is the Random Forest, with a 0.85 accuracy. This permited as to see which models will have a better performance with the data from the previous years, when we don't have the current year information.

In [ ]:
plt.boxplot(cross_val_scores, labels=['Random Forest', 'KNN', 'Multinomial NB', 'MLP'])
plt.title('Cross-Validation Scores')
plt.ylabel('Accuracy')
plt.show()

The Random Forest Classifier seems to be performing better than the other classifiers.

In the next chapter we will explore using a rolling window to use past years for the predictions

## Rolling window

### Rolling window functions

We will use a rolling window to predict the next season. We will use the last 3 seasons to predict the next one.

In [ ]:
# Define the rolling window size (number of past years to consider)
ROLLING_WIN_SIZE = 3  # You can adjust this value
PREDICTION_YEAR = 11

historical_data_df = teams_df[teams_df['year'] < PREDICTION_YEAR]

def create_rolling_window_dataset(df, window_size):
    rolling_window_data = []
    for tmID, group in df.groupby('tmID'):
        group = group.sort_values(by='year')
        for i in range(len(group) - window_size + 1):
            window = group.iloc[i:i + window_size]
            rolling_window_data.append(window)
    return rolling_window_data

def generate_weights(window_size):
    weights = np.linspace(0.1, 1, window_size)
    weights /= weights.sum()  # Normalize weights to ensure they sum to 1
    return weights

Create the rolling window dataset.
In this dataset, three years are combined, with more weight given to the most recent year, and combining that historic data with the target variable (playoff) from the target year (the n+1 year)

In [ ]:
def rolling_window_df():
    # Create the rolling window datasets
    rolling_window_historic = create_rolling_window_dataset(historical_data_df,  ROLLING_WIN_SIZE)

    # Define weights (you can adjust these weights as needed)
    weights = generate_weights(ROLLING_WIN_SIZE)

    weighted_features = ['o_fta', 'o_3pm', 'o_oreb', 'o_asts', 'o_stl', 'o_to', 'o_blk', 'd_3pa', 'd_oreb', 'd_dreb', 'd_asts', 'd_stl', 'd_to', 'd_blk', 'attend']

    if USE_MERGED_DATA:
        weighted_features += ['blocks_weighted_mean', 'threeAttempted_weighted_mean', 'FG%_weighted_mean', 'FT%_weighted_mean', 'Avg_Assists_Per_Game_weighted_mean', 'Durability_Ratio_weighted_mean', 'POSITION_METRIC_weighted_mean', 'num_trophies']

    # Create a new dataframe to store the rolling window data
    rolling_data = pd.DataFrame(columns=teams_df.columns)

    for window in rolling_window_historic:
        #For each window, we will add a line to the rolling_data dataframe, as follows:
        #1. Create a new empty line for the rolling_data dataframe
        rolling_data_line = pd.DataFrame(columns=teams_df.columns)

        #2. Get the weighted average of the weighted features
        #Note: The weighted average is calculated as follows:
        #weighted average = sum of (weight * feature value) / sum of weights
        #For example, if the weights are [0.2, 0.3, 0.5] and the feature values are [10, 20, 30], then the weighted average is:
        #weighted average = (0.2 * 10 + 0.3 * 20 + 0.5 * 30) / (0.2 + 0.3 + 0.5) = 24
        for feature in weighted_features:
            weighted_average = np.average(window[feature].values, weights=weights)
            rolling_data_line[feature] = [weighted_average]

        #3. Join that data with the data from the following year (the year we want to predict)
        #Note: The data from the following year will be used as the target label, get next year from the teams_df dataframe
        next_year = window['year'].max() + 1
        next_year_data = teams_df[(teams_df['tmID'] == window['tmID'].values[0]) & (teams_df['year'] == next_year)]

        #Add year, confID, tmID, and playoff columns
        try:
            rolling_data_line['year'] = next_year
            rolling_data_line['tmID'] = window['tmID'].values[0]
            rolling_data_line['confID'] = next_year_data['confID'].values[0]
            rolling_data_line['playoff'] = next_year_data['playoff'].values[0]
        except:
            #If there is no data for the next year, skip this line
            #print('No data for next year')
            continue
        
        #4. Concat the data to the rolling_data dataframe
        rolling_data = pd.concat([rolling_data, rolling_data_line])

    rolling_data = rolling_data.reset_index(drop=True)
    return rolling_data

### Random Forest with Rolling window

In [ ]:
rolling_data = rolling_window_df()

y_train = rolling_data[rolling_data['year'] < PREDICTION_YEAR]['playoff'].astype(int)
y_test = rolling_data[rolling_data['year'] == PREDICTION_YEAR]['playoff'].astype(int)
confIDs = rolling_data[rolling_data['year'] == PREDICTION_YEAR]['confID']
X_train = rolling_data[rolling_data['year'] < PREDICTION_YEAR].drop(['playoff', 'year', 'tmID', 'firstRound', 'semis', 'finals'], axis=1)
X_test = rolling_data[rolling_data['year'] == PREDICTION_YEAR].drop(['playoff', 'year', 'tmID', 'firstRound', 'semis', 'finals'], axis=1)

if USE_MERGED_DATA:
    X_train = X_train.drop(['W_post', 'L_post'], axis=1)
    X_test = X_test.drop(['W_post', 'L_post'], axis=1)

clf = RandomForestClassifier(n_estimators=100)

X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])

display(X_train.head())

y_pred = print_results(clf, X_train, X_test, y_train, y_test)

force_qualify_8_teams(y_pred, y_test, confIDs)

scores = cross_validation(clf, X, y)

## Grid Search

### Random Forest

Now, we will try to find what are the best parameters for the Random Forest, KNN and M Naive Bayes models, using Grid Search, and tune this models to see if we can improve the results.

In [ ]:
#use grid search to find the best parameters
cross_val_scores = []
best_clf = {}

parameters = {'n_estimators': [10, 50, 100], 'max_depth': [9, 11, 13], 'max_features': ['sqrt', 'log2', 0.2], 'criterion': ['gini', 'entropy']}
clf = RandomForestClassifier()
grid_search = GridSearchCV(clf, parameters, cv=5, scoring='accuracy', verbose=3, n_jobs=-1)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)
print(grid_search.best_estimator_)
clf = grid_search.best_estimator_
best_clf['Random Forest'] = (grid_search.best_score_, grid_search.best_estimator_)

In [ ]:
# Use the best parameters to train the model
y_pred = print_results(clf, X_train, X_test, y_train, y_test)
force_qualify_8_teams(y_pred, y_test, confIDs)
scores = cross_validation(clf, X, y)
cross_val_scores.append(scores)

### KNN

In [ ]:
# Use grid search to find the best parameters
parameters = {'n_neighbors': [3, 5, 7, 9, 11, 13], 'weights': ['uniform', 'distance']}
clf = KNeighborsClassifier()
grid_search = GridSearchCV(clf, parameters, cv=5, scoring='accuracy', verbose=3, n_jobs=-1)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)
print(grid_search.best_estimator_)
clf = grid_search.best_estimator_
best_clf['KNN'] = (grid_search.best_score_, grid_search.best_estimator_)

In [ ]:
#use the best parameters to train the model
y_pred = print_results(clf, X_train, X_test, y_train, y_test)
force_qualify_8_teams(y_pred, y_test, confIDs)
scores = cross_validation(clf, X, y)
cross_val_scores.append(scores)

### Multinomial NaiveBayes

In [ ]:
#use grid search to find the best parameters
parameters = {'alpha': [0.1, 0.5, 1, 2, 5, 10]}
clf = MultinomialNB()
grid_search = GridSearchCV(clf, parameters, cv=5, scoring='accuracy', verbose=3, n_jobs=-1)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)
print(grid_search.best_estimator_)
clf = grid_search.best_estimator_
best_clf['Multinomial NB'] = (grid_search.best_score_, grid_search.best_estimator_)

In [ ]:
#use the best parameters to train the model
y_pred = print_results(clf, X_train, X_test, y_train, y_test)
force_qualify_8_teams(y_pred, y_test, confIDs)
scores = cross_validation(clf, X, y)
cross_val_scores.append(scores)

### Plotting the results

In [ ]:
#plot the cross validation scores
plt.boxplot(cross_val_scores, labels=['Random Forest', 'KNN', 'Multinomial NB'])
plt.title('Cross-Validation Scores')
plt.ylabel('Accuracy') 
plt.show()

After getting our best classifier, we will try different numbers of ROLLING WINDOW to see if we can improve the results.

In [ ]:
# Initialize variables to store the best score and estimator
best_score = -1
best_estimator = None
best_clf_name = None

# Iterate over the best_clf dictionary
for clf_name, (score, estimator) in best_clf.items():
    # If this classifier's score is higher than the current best score, update the best score and estimator
    if score > best_score:
        best_score = score
        best_estimator = estimator
        best_clf_name = clf_name

print(f"The best classifier is {best_clf_name} with a score of {best_score}")

cross_val_scores = []
for i in range(1, 10):
    ROLLING_WIN_SIZE = i
    rolling_data = rolling_window_df()

    y_train = rolling_data[rolling_data['year'] < PREDICTION_YEAR]['playoff'].astype(int)
    y_test = rolling_data[rolling_data['year'] == PREDICTION_YEAR]['playoff'].astype(int)
    confIDs = rolling_data[rolling_data['year'] == PREDICTION_YEAR]['confID']
    X_train = rolling_data[rolling_data['year'] < PREDICTION_YEAR].drop(['playoff', 'year', 'tmID', 'firstRound', 'semis', 'finals'], axis=1)
    X_test = rolling_data[rolling_data['year'] == PREDICTION_YEAR].drop(['playoff', 'year', 'tmID', 'firstRound', 'semis', 'finals'], axis=1)

    if USE_MERGED_DATA:
        X_train = X_train.drop(['W_post', 'L_post'], axis=1)
        X_test = X_test.drop(['W_post', 'L_post'], axis=1)

    clf = best_estimator

    X = pd.concat([X_train, X_test])
    y = pd.concat([y_train, y_test])

    y_pred = print_results(clf, X_train, X_test, y_train, y_test, False)

    force_qualify_8_teams(y_pred, y_test, confIDs, False)

    scores = cross_validation(clf, X, y, False)
    cross_val_scores.append(scores)

plt.boxplot(cross_val_scores, labels=['1', '2', '3', '4', '5', '6', '7', '8', '9'])